In [2]:
# Please write a python function that takes three inputs as and achieves the task defined below:

# calculatePercentAreaOfPolygonAInPolygonB(input_geodatabase, fcPolygon1, fcPolygon2):

# Given an input polygon feature class, fcPolygonA (e.g., parks), and a second feature class, fcPolygonB (e.g., block_groups) :

# Calculate the percentage of the area of the first polygon features (fcPolygonA ) in the second polygon features (fcPolygonB),
# and append the percentage of park area into a new field in the block groups feature class
# Make sure that your area calculations are as accurate as possible. 

In [9]:
import arcpy
arcpy.env.overwriteOutput = True

In [23]:
def calculatePercentAreaOfPolygonAInPolygonB(input_geodatabase, fcPolygon1, fcPolygon2):
    try:
        ## Update workspace
        arcpy.env.workspace = input_geodatabase
        
        ## Create field name objects
        cbg_area_field = 'cbg_area'
        park_area_field = 'park_area'
        park_pct_field = 'park_pct'
        fcOverlap = 'park_overlap'
        
        ## Add new fields used in calculations to fcPolygon2
        arcpy.AddField_management(fcPolygon2, cbg_area_field, "DOUBLE")
        arcpy.AddField_management(fcPolygon2, park_area_field, "DOUBLE")
        arcpy.AddField_management(fcPolygon2, park_pct_field, 'DOUBLE')
        
        ## Calculate area of block groups, perform an intersect analysis, and then calculate area of intersect.
        arcpy.CalculateGeometryAttributes_management(fcPolygon2, [[cbg_area_field, "AREA_GEODESIC"]], "MILES_US")
        arcpy.Intersect_analysis([fcPolygon2, fcPolygon1], fcOverlap)
        arcpy.CalculateGeometryAttributes_management(fcOverlap, [[park_area_field, "AREA_GEODESIC"]], "MILES_US")

        ## Create dictionary of area of Polygon1 by FIPS code, used for combining with Polygon2
        cbg_dict = {}
        with arcpy.da.SearchCursor(fcParkOverlap, ["FIPS", park_area_field]) as cursor:
            for row in cursor:
                geoid = row[0]
                if geoid in cbg_dict.keys():
                    cbg_dict[geoid] += row[1]
                else:
                    cbg_dict[geoid] = row[1]
        
        ## Match park area from fcParkOverlap with FIPS in Polygon2
        with arcpy.da.UpdateCursor(fcPolygon2, ["FIPS", park_area_field]) as cursor:
            for row in cursor:
                if row[0] in cbg_dict.keys():
                    row[1] = cbg_dict[row[0]]
                else:
                    row[1] = 0
                cursor.updateRow(row)
        ## Calculate percent overlap and input into park_pct_field     
        arcpy.CalculateField_management(fcPolygon2, park_pct_field, "!park_area!/!cbg_area!", "PYTHON3")
        
    except:
        # By default any other errors will be caught here
        e = sys.exc_info()[1]
        print(e.args[0])

In [26]:
workspace = r'E:\My Drive\School\College\06 Grad Full\Semester 2\Geospatial Programming\Quizzes\Quiz 5\Quiz 5/quiz5.gdb'

calculatePercentAreaOfPolygonAInPolygonB(workspace, 'parks_test1', 'block_groups_test1')

In [10]:
fcPolygon1 = 'parks'
fcPolygon2 = 'block_groups'
fcParkOverlap = 'park_overlap'

arcpy.AddField_management(fcPolygon2, "cbg_area", "DOUBLE")
arcpy.AddField_management(fcPolygon2, "park_area", "DOUBLE")
arcpy.AddField_management(fcPolygon2, 'park_pct', 'DOUBLE')

<Result 'block_groups'>

In [11]:
arcpy.CalculateGeometryAttributes_management(fcPolygon2, [["cbg_area", "AREA_GEODESIC"]], "MILES_US")
arcpy.Intersect_analysis([fcPolygon2, fcPolygon1], "park_overlap")
arcpy.CalculateGeometryAttributes_management('park_overlap', [["park_area", "AREA_GEODESIC"]], "MILES_US")

<Result 'park_overlap'>

In [14]:
park_area_field = 'park_area'

cbg_dict = {}
with arcpy.da.SearchCursor(fcParkOverlap, ["FIPS", park_area_field]) as cursor:
    for row in cursor:
        geoid = row[0]
        if geoid in cbg_dict.keys():
            cbg_dict[geoid] += row[1]
        else:
            cbg_dict[geoid] = row[1]

In [16]:
with arcpy.da.UpdateCursor(fcPolygon2, ["FIPS", park_area_field]) as cursor:
    for row in cursor:
        if row[0] in cbg_dict.keys():
            row[1] = cbg_dict[row[0]]
        else:
            row[1] = 0
        cursor.updateRow(row)

In [18]:
park_pct_field = 'park_pct'
arcpy.CalculateField_management(fcPolygon2, park_pct_field, "!park_area!/!cbg_area!", "PYTHON3")

<Result 'block_groups'>